In [ ]:
import numpy as np 
import tensorflow as tf
import os 
import sys 
main_pth = '/home/jy/Documents/fnirs/treatment_response/fnirs-depression-deeplearning/'
sys.path.append(main_pth)
os.chdir(main_pth)
from LOO_nested_CV_model import build_model


model_name = 'gnn_transformer'
config_file_name = 'pretreatment_response'
msg = 'loocv_v1'
TrainModel = build_model(model_name, config_file_name,msg)


In [ ]:
from utils.utils_mine import stratified_LOO_nested_CV

data = TrainModel.data
label = TrainModel.label
adj = TrainModel.adj
current_loo = 0
k = 0
num_of_k_fold = 5
classifier = TrainModel.model
X_train, Y_train, X_val, Y_val, X_test, Y_test, adj_train, adj_val, adj_test = stratified_LOO_nested_CV(data, label, k=k, num_of_k_fold=num_of_k_fold, current_loo=current_loo, adj=adj)

print(f'X_train: {X_train.shape}')
print(f'X_val: {X_val.shape}')
print(f'X_test: {X_test.shape}')

In [ ]:
import shap
def compute_data_metrics(classifier, input_data, input_adj, y_true):
    # Assuming the TensorFlow  is stored in the variable `model`
    y_train_pred = classifier.model.predict([input_data, input_adj])
    y_train_pred_argmax = np.argmax(y_train_pred, axis=1)
    y_train_true = np.argmax(y_true, axis=1)
    from utils.utils_mine import get_metrics
    return get_metrics(y_train_pred_argmax, y_train_true)

for subject in range(X_train.shape[0]):
    for k_fold in range(num_of_k_fold):
        X_train, Y_train, X_val, Y_val, X_test, Y_test, adj_train, adj_val, adj_test = stratified_LOO_nested_CV(data, label, k=k_fold, num_of_k_fold=num_of_k_fold, current_loo=subject, adj=adj)

        checkpoint_path = f'results/gnn_transformer/prognosis/pretreatment_response/loocv_v1l1_rate_0.01_l2_rate_0.01_d_model_16_batch_size_64_n_layers_6/LOO_nested_CV/LOO_{subject}/stratified_nested_5_CV_fold-{k_fold}/fold-best-checkpoint'
        classifier.model.load_weights(checkpoint_path)

        
        # NOTE TO DO: why the train metrics are very bad? see sub-0 fold-0/1 
        # train_metrics = compute_data_metrics(classifier, X_train, adj_train, Y_train)
        # val_metrics = compute_data_metrics(classifier, X_val, adj_val, Y_val)
        # print(f"sub - {subject} - fold - {k_fold} - train metrics - {train_metrics}")
        # print(f"sub - {subject} - fold - {k_fold} - val metrics - {val_metrics}")
        
        
        import shap 
        explainer = shap.GradientExplainer(classifier.model, [X_train, adj_train])
        shap_values = explainer.shap_values([X_test, adj_test])
        break
    break

In [ ]:
from sklearn.metrics import accuracy_score

y_true = [0, 1, 0, 1, 1]  # Actual labels
y_pred = [0, 1, 1, 1, 0]  # Predicted labels

accuracy = accuracy_score(y_true, y_pred)
print(f"Accuracy: {accuracy}")
